In [0]:
# Gdrive Linking and symbolic
import os
from google.colab import drive
drive.mount('/content/drive')
os.symlink('/content/drive/My Drive/LisaSSD', '/content/project')

In [0]:
# Only for resume session
%cd /content/project/models/research
!export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim
!protoc ./object_detection/protos/*.proto --python_out=.
!sudo python setup.py install

%cd models/research/slim
!sudo python setup.py install

In [0]:
# LISA dataset prepare
%cd /content/project
!wget http://cvrr.ucsd.edu/LISA/Datasets/signDatabasePublicFramesOnly.zip
!git clone https://github.com/sooxt98/Lisa-MobileNetV3
!mkdir lisa
!unzip signDatabasePublicFramesOnly.zip -d lisa

# tensor setup for LISA
!git clone https://github.com/tensorflow/models/
!cd models/research && export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim
!cd models/research && protoc ./object_detection/protos/*.proto --python_out=.
!cd models/research && sudo python setup.py install
!cd models/research/slim && sudo python setup.py install

In [0]:
# generate TFrecord
!mkdir output
!cd Lisa-MobileNetV3 && python ./create_lisa_tfrecords.py --data_dir=../lisa --output_dir=../output --label_map_path=./lisa_label_map.pbtxt

In [0]:
# Tensorboard <3
%load_ext tensorboard

In [0]:
%tensorboard --logdir /content/project/output

In [0]:
# Let's train with GPU
%%shell
PIPELINE_CONFIG_PATH=/content/project/Lisa-MobileNetV3/ssdlite_mobilenet_v3_large_320x320_coco.config
MODEL_DIR=/content/project/output
NUM_TRAIN_STEPS=50000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1
python /content/project/models/research/object_detection/model_main.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    --num_train_steps=${NUM_TRAIN_STEPS} \
    --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES \
    --alsologtostderr

In [0]:
# misc
!rm -rf Lisa-MobileNetV3
!git clone https://github.com/sooxt98/Lisa-MobileNetV3